In [266]:
import numpy as np
np.set_printoptions(suppress=True)
from load_measurements import load_measurements
from aggregate_measurements import aggregate_measurements
from print_statistics import print_statistics
# print(f"{0.809:.2%}")

In [2]:
def load_measurements2(filename, fmode):
    # abspath = os.path.dirname(os.path.abspath(__file__))
    # path = abspath + "\\" + filename
    data = [row.split(",") for row in open(filename, "r")]
    data = np.array(data,dtype=float)

    if fmode == "drop":
        # remove all rows that contain a corrupted measurement:
        data = data[np.all(data != -1, axis=1)]

    if fmode == "forward fill" and np.all(data[0] != -1): num = -1
    elif fmode == "backward fill" and np.all(data[-1] != -1): num = 1
    else:
        data = data[np.all(data != -1, axis=1)]
        print("errmsg")

    for x, y in np.argwhere(data == -1):
        data[x][y] = data[x + num][y]

    tvec = data[:,:6]
    data = data[:,6:]
    return tvec, data

In [264]:
def print_statistics2(_, data):
    zones = np.arange(1,5)[:, None]
    stats = np.array([
        np.min(data, axis=0),
        np.quantile(data, 0.25, axis=0),
        np.quantile(data, 0.50, axis=0),
        np.quantile(data, 0.75, axis=0),
        np.max(data, axis=0)
    ]).T
    stats2 = np.array([
        "All",
        np.min(data),
        np.quantile(data, 0.25),
        np.quantile(data, 0.50),
        np.quantile(data, 0.75),
        np.max(data)
    ])

    statistics = np.hstack((zones, stats))
    statistics = np.vstack((statistics, stats2))
    
    splitline = "-"*65
    headers = ["Zone", "Minimum", "1. quart", "2. quart", "3. quart", "Maximum"]

    print(splitline)
    print(*[f"{header:^10}" for header in headers])
    print(splitline)
    [print(*[f"{header:^10}" for header in row]) for row in statistics]
    print(splitline)

In [265]:
print_statistics(*load_measurements("testdata2.csv", "forward fill"))

-----------------------------------------------------------------
   Zone     Minimum    1. quart   2. quart   3. quart   Maximum  
-----------------------------------------------------------------
   1.0        2.0        2.75       4.0        5.5        7.0    
   2.0        3.0        3.0        4.5        6.0        6.0    
   3.0        1.0        1.0        2.5        5.25       9.0    
   4.0        0.0        3.75       5.5        6.5        8.0    
   All        0.0        2.75       4.5        6.0        9.0    
-----------------------------------------------------------------


In [268]:
tvec, data = load_measurements("2008.csv", "drop")

errmsg


In [357]:
def aggregate_measurements(tvec, data, period):
    tvec_a = None
    col = 0
    if period == "hour":
        col = 3
    elif period == "day":
        col = 2
    elif period == "month":
        col = 1
    elif period == "hour of the day":
        col = 3

    uniq = np.unique(tvec[:,col])
    mask = None
    for n in uniq:
        mask = tvec[:,col] == n
        if period != "hour of the day":
            list1 = [0]*6
            list1[col] = n
            print(list1)
            print(f"{int(n):<5}", *[f"{round(num,2):<12}" for num in np.sum(data[mask], axis=0)])
        else:
            2
            # num = np.round(np.mean(data[mask], axis=0), 2)
            # print(*[f"{n:<6}" for n in num])
            # print(f"{int(n):<5}", *[f"{round(num,2):<12}" for num in np.sum(data[mask], axis=0)])
    
    # return tvec_a, data
    

# print(tvec[:,:(3+1)])
aggregate_measurements(tvec, data, "month")

[0, 1.0, 0, 0, 0, 0]
1     61761.0      62911.0      312175.0     1086156.4   
[0, 2.0, 0, 0, 0, 0]
2     40192.0      56646.0      255918.0     822177.5    
[0, 3.0, 0, 0, 0, 0]
3     63110.0      79277.0      279670.0     926517.2    
[0, 4.0, 0, 0, 0, 0]
4     47080.0      70871.0      295678.0     803505.4    
[0, 5.0, 0, 0, 0, 0]
5     53084.0      58907.0      290620.0     762024.7    
[0, 6.0, 0, 0, 0, 0]
6     68728.0      63449.0      290103.0     715737.4    
[0, 7.0, 0, 0, 0, 0]
7     47313.0      44416.0      227228.0     591297.6    
[0, 8.0, 0, 0, 0, 0]
8     3873.0       22264.0      79665.0      205648.2    
[0, 9.0, 0, 0, 0, 0]
9     52376.0      44590.0      284282.0     711118.3    
[0, 10.0, 0, 0, 0, 0]
10    44093.0      63228.0      275887.0     844942.0    
[0, 11.0, 0, 0, 0, 0]
11    58067.0      55887.0      280615.0     998498.8    
[0, 12.0, 0, 0, 0, 0]
12    45107.0      39459.0      307346.0     947242.0    
